# MGMTMSA 403 Homework 2: Portfolio Optimization


## Pre-processing step: Estimate expected returns and covariance

In [11]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

# Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"]

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) # stores the tickets of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) # collect index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',', skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1, n))
for stock in range(n):
    for month in range(d-1):
        returns[month, stock] = prices[month+1, stock]/prices[month, stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns, axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))


We use the following formulation for all Markowitz problems:

\begin{align}
{\text{min}} \;\; & \sum_{i=1}^n \sum_{j=1}^n w_{i} w_{j}C_{ij} \\
\text{s.t. } & \sum_{i=1}^n w_{i} r_{i} \geq \rho, \\
& \sum_{i=1}^n w_{i} = 1, \\
& w_{i} \geq 0, \text{ } i = 1, \ldots, n. \\
\end{align}

## Model 1

In [12]:
# initialize model
mod_1 = Model()

# initialize weight vector to return
w_1 = mod_1.addVars(4)

# minimum return constraint
mod_1.addConstr(sum(w_1[i] * avg_return[tick] for i, tick in enumerate(tickind)) >= 0.005)

# total investment amount constraint
mod_1.addConstr(sum(w_1[i] for i in range(4)) == 1)

# non-negativity constraint
for i in range(4):
    mod_1.addConstr(w_1[i] >= 0)
    
# construct objective (minimize risk)
mod_1.setObjective(sum(w_1[i] * w_1[j] * C[tick_i][tick_j] for i, tick_i in enumerate(tickind)
                       for j, tick_j in enumerate(tickind)), GRB.MINIMIZE)


In [13]:
# update and solve model
mod_1.update()
mod_1.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x70c79b8a
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87864836e+05 -1.87864836e+05  3.50e+03 1.91e-07  1.00e+06     0s
   1   5.03195868e+

In [14]:
# display optimal risk value
opt_val_1 = mod_1.objval
print("Minimum risk:", opt_val_1)


Minimum risk: 0.0001774932651657861


In [15]:
# display weights

print('Weight vector:')
for i in range(4):
    print(w_1[i])
print()
    
print('Investments:')
tick4 = ["MSFT","GS","PG","SCHP"]

for i in range(4):
    stock = tick4[i]
    weight = w_1[i].x
    print(f'{stock}: {weight * 100:<.2f}%')


Weight vector:
<gurobi.Var C0 (value 0.23711749137102442)>
<gurobi.Var C1 (value 0.025859835330683586)>
<gurobi.Var C2 (value 1.5749710132194975e-09)>
<gurobi.Var C3 (value 0.737022671723319)>

Investments:
MSFT: 23.71%
GS: 2.59%
PG: 0.00%
SCHP: 73.70%


## Model 2

In [16]:
# initialize model
mod_2 = Model()

# initialize weight vector to return
w_2 = mod_2.addVars(390)

# minimum return constraint
mod_2.addConstr(sum(w_2[i] * avg_return[i] for i in range(390)) >= 0.005)

# total investment amount constraint
mod_2.addConstr(sum(w_2[i] for i in range(390)) == 1)

# non-negativity constraint
for i in range(390):
    mod_2.addConstr(w_2[i] >= 0)
    
# construct objective (minimize risk)
mod_2.setObjective(sum(w_2[i] * w_2[j] * C[i][j] for i in range(390) for j in range(390)),
                   GRB.MINIMIZE)


In [17]:
# update and solve model
mod_2.update()
mod_2.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0x8a1af9c5
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.03s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0s


In [18]:
# display optimal risk value
opt_val_2 = mod_2.objval
print("Minimum risk:", opt_val_2)


Minimum risk: 2.8785375543938808e-05


## Model 3

Allowing for investment in only 4 or less stocks, we get the following modified setup:

\begin{align}
{\text{min}} \;\; & \sum_{i=1}^n \sum_{j=1}^n w_{i} w_{j}C_{ij} \\
\text{s.t. } & \sum_{i=1}^n w_{i} r_{i} \geq \rho, \\
& \sum_{i=1}^n w_{i} = 1, \\
& \sum_{i=1}^n w_{i} x_{i} = 1, \\
& \sum_{i=1}^n x_{i} \leq 4, \\
& w_{i} \geq 0, \text{ } i = 1, \ldots, n. \\
\end{align}

In [19]:
# initialize model
mod_3 = Model()

# initialize weight vector to return
w_3 = mod_3.addVars(390)
x = mod_3.addVars(390, vtype = GRB.BINARY)

# minimum return constraint
mod_3.addConstr(sum(w_3[i] * avg_return[i] for i in range(390)) >= 0.005)

# total investment amount constraints
mod_3.addConstr(sum(w_3[i] * x[i] for i in range(390)) == 1)
mod_3.addConstr(sum(w_3[i] for i in range(390)) == 1)

# only invest in 4 stocks
mod_3.addConstr(sum(x[i] for i in range(390)) <= 4)

# non-negativity constraint
for i in range(390):
    mod_3.addConstr(w_3[i] >= 0)
    
# construct objective (minimize risk)
mod_3.setObjective(sum(w_3[i] * w_3[j] * C[i][j] for i in range(390) for j in range(390)),
                   GRB.MINIMIZE)


In [20]:
# update and solve model
mod_3.update()
mod_3.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 393 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0x859e9ed2
Model has 76245 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1174 rows, 1170 columns, 4290 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0021239

Root relaxation: objective 2.878501e-05, 1586 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Obje

In [21]:
# display optimal risk value
opt_val_3 = mod_3.objval
print("Minimum risk:", opt_val_3)


Minimum risk: 6.75347076072811e-05


In [22]:
# display optimal portfolio
for i in range(390):
    if x[i].x != 0:
        stock = tickers[i]
        weight =  w_3[i].x
        print(f'{stock}: {weight * 100:<.2f}%')


CME: 12.64%
LLY: 7.55%
NVDA: 4.38%
BND: 75.44%


## Model 3, Question 3a

Same model, but we impose a time limit of 30 seconds:

In [23]:
# initialize model
mod_3a = Model()

# initialize weight vector to return
w_3a = mod_3a.addVars(390)
x_a = mod_3a.addVars(390, vtype = GRB.BINARY)

# minimum return constraint
mod_3a.addConstr(sum(w_3a[i] * avg_return[i] for i in range(390)) >= 0.005)

# total investment amount constraints
mod_3a.addConstr(sum(w_3a[i] * x_a[i] for i in range(390)) == 1)
mod_3a.addConstr(sum(w_3a[i] for i in range(390)) == 1)

# only invest in 4 stocks
mod_3a.addConstr(sum(x_a[i] for i in range(390)) <= 4)

# non-negativity constraint
for i in range(390):
    mod_3a.addConstr(w_3a[i] >= 0)
    
# impose time limit
mod_3a.Params.TimeLimit = 30.0
    
# construct objective (minimize risk)
mod_3a.setObjective(sum(w_3a[i] * w_3a[j] * C[i][j] for i in range(390) for j in range(390)),
                    GRB.MINIMIZE)


Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [24]:
# update and solve model
mod_3a.update()
mod_3a.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 393 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0x859e9ed2
Model has 76245 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1174 rows, 1170 columns, 4290 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0021239

Root relaxation: objective 2.878501e-05, 1586 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Obje

In [25]:
# display optimal portfolio
for i in range(390):
    if x_a[i].x != 0:
        stock = tickers[i]
        weight =  w_3a[i].x
        print(f'{stock}: {weight * 100:<.2f}%')
        

CME: 12.64%
LLY: 7.55%
NVDA: 4.38%
BND: 75.44%


## Model 3, Question 3b

Same model, but we impose an optimality gap of 10% (0.1):

In [26]:
# initialize model
mod_3b = Model()

# initialize weight vector to return
w_3b = mod_3b.addVars(390)
x_b = mod_3b.addVars(390, vtype = GRB.BINARY)

# minimum return constraint
mod_3b.addConstr(sum(w_3b[i] * avg_return[i] for i in range(390)) >= 0.005)

# total investment amount constraints
mod_3b.addConstr(sum(w_3b[i] * x_b[i] for i in range(390)) == 1)
mod_3b.addConstr(sum(w_3b[i] for i in range(390)) == 1)

# only invest in 4 stocks
mod_3b.addConstr(sum(x_b[i] for i in range(390)) <= 4)

# non-negativity constraint
for i in range(390):
    mod_3b.addConstr(w_3b[i] >= 0)
    
# set optimality gap limit
mod_3b.Params.MIPGap = 0.1
    
# construct objective (minimize risk)
mod_3b.setObjective(sum(w_3b[i] * w_3b[j] * C[i][j] for i in range(390) for j in range(390)),
                    GRB.MINIMIZE)


Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001


In [27]:
# update and solve model
mod_3b.update()
mod_3b.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 393 rows, 780 columns and 1560 nonzeros
Model fingerprint: 0x859e9ed2
Model has 76245 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 1174 rows, 1170 columns, 4290 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0021239

Root relaxation: objective 2.878501e-05, 1586 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Obje

In [28]:
# display optimal portfolio
for i in range(390):
    if x_b[i].x != 0:
        stock = tickers[i]
        weight =  w_3b[i].x
        print(f'{stock}: {weight * 100:<.2f}%')


CME: 12.64%
LLY: 7.55%
NVDA: 4.38%
BND: 75.44%
